In [1]:
import os
import numpy as np
import pandas as pd
import folium
from folium import plugins
import geopandas
import argparse
import glob
import gpxpy
import geojson
import webbrowser
import numpy as np
import matplotlib.cm as cm
from scipy.signal import medfilt


In [2]:
# functions

def calc_dist_between_one_point_to_all_points(lon0, lat0, lon_all, lat_all): 
    lat0_rad    = np.radians(lat0)
    lon0_rad    = np.radians(lon0)
    lat_all_rad = np.radians(lat_all)
    lon_all_rad = np.radians(lon_all)

    delta_lat = lat_all_rad - lat0_rad
    delta_lon = lon_all_rad - lon0_rad

    # apply haversine formula
    a = (np.sin(delta_lat/2.0))**2.0 + np.cos(lat0_rad)*np.cos(lat0_rad)*((np.sin(delta_lon/2.0))**2.0)
    c = 2.0*np.arctan2(np.sqrt(a), np.sqrt(1.0-a))

    r = 6371.0*1000.0
    dist_all = r*c
    
    return(dist_all)


def calc_dist_between_two_coords(lon0, lat0, lon1, lat1): 
    lat0_rad = np.radians(lat0)
    lat1_rad = np.radians(lat1)
    lon0_rad = np.radians(lon0)
    lon1_rad = np.radians(lon1)

    delta_lat = lat1_rad - lat0_rad
    delta_lon = lon1_rad - lon0_rad

    # apply haversine formula
    a = (np.sin(delta_lat/2.0))**2.0 + np.cos(lat0_rad)*np.cos(lat1_rad)*((np.sin(delta_lon/2.0))**2.0)
    c = 2.0*np.arctan2(np.sqrt(a), np.sqrt(1.0-a))

    r = 6371.0*1000.0
    dist = r*c
    
    return(dist)

def calc_dist_from_coords(p1, p2): # distance between p1 and p2 [lat,lon] (in deg)
    lat1 = np.radians(p1[0])
    lat2 = np.radians(p2[0])
    lon1 = np.radians(p1[1])
    lon2 = np.radians(p2[1])

    delta_lat = lat2-lat1
    delta_lon = lon2-lon1

    # Haversine formula
    a = np.power(np.sin(delta_lat/2.0), 2)+np.cos(lat1)*np.cos(lat2)*np.power(np.sin(delta_lon/2.0), 2)
    c = 2.0*np.arctan2(np.sqrt(a), np.sqrt(1.0-a))

    dist = 6371e3*c

    return(dist)

def calc_dist_from_coordsPoint2Line(p0, p1, p2): # distance from p0 to line defined by p1 and p2 [lat,lon] (in deg)
    # Mercator projection
    P0 = np.array([np.radians(p0[1]), np.arcsinh(np.tan(np.radians(p0[0])))])*6371e3
    P1 = np.array([np.radians(p1[1]), np.arcsinh(np.tan(np.radians(p1[0])))])*6371e3
    P2 = np.array([np.radians(p2[1]), np.arcsinh(np.tan(np.radians(p2[0])))])*6371e3

    # distance from point to line
    dist = abs((P2[1]-P1[1])*P0[0]-(P2[0]-P1[0])*P0[1]+P2[0]*P1[1]-P2[1]*P1[0])/np.sqrt(np.power(P2[1]-P1[1], 2)+np.power(P2[0]-P1[0], 2)) # (from https://en.wikipedia.org/wiki/Distance_from_a_point_to_a_line#Line_defined_by_two_points)

    return(dist)

def RDP(data, epsilon): # Ramer–Douglas–Peucker algorithm
    if epsilon <= 0:
        return(data)

    dist_max_rdp = 0
    index = 0

    for i in np.arange(1, data.shape[0]):
        dist = calc_dist_from_coordsPoint2Line(data[i, :2], data[0, :2], data[-1, :2]) # needs a 2D projection, does not work with cross-track distance

        if (dist > dist_max_rdp):
            index = i
            dist_max_rdp = dist

    if (dist_max_rdp > epsilon):
        tmp1 = RDP(data[:index+1, :], epsilon)
        tmp2 = RDP(data[index:, :], epsilon)

        data_new = np.vstack((tmp1[:-1], tmp2))
    else:
        data_new = np.vstack((data[0, :], data[-1, :]))

    return(data_new)



In [3]:
dir_gpx     = 'data/gpx' 
dir_geojson = 'data/geojson'

dir_work = '/home/craigmatthewsmith/gps_tracks'
os.chdir(dir_work)
#os.getcwd()

#gpx_file_temp = os.path.join(dir_work, 'data_input_gpx/Afternoon_Run55.gpx')
#print(os.path.isfile(gpx_file_temp))

ingest_file_list = glob.glob(os.path.join(dir_gpx, '*.gpx'))
n_files = len(ingest_file_list)
print('found %s files to process ' %(n_files))                              
#print('found %s files' %(n_files))                              
#geojson_file = os.path.join(data_geojson, '2020-03-22_15-06.geojson')
#os.path.isfile(geojson_file)



found 263 files to process 


In [4]:
use_RDP  = True
epsilon  = 1.0 # rdp thinning 
#dist_min_aggregate_points = 1.0 
dist_min_aggregate_points = 3.0 
#dist_min_aggregate_points = 10.0 

#  1.0, reduced points from 75752 to 65536, 9.5 M master.geojson file size 
#  3.0, reduced points from 75752 to 44553, 6.2 M master.geojson file size  
# 10.0, reduced points from 75752 to 19330, 2.5 M master.geojson file size

dist_max_between_points_to_make_line = 100.0 # dont plot lines this far away

# thin2 deprecated only
dist_min_adjacent_points_to_line = 30.0 


In [5]:
lat_all = []
lon_all = []
ele_all = []
dt_all  = []

f = 100
#for f in range(0, 20, 1):
#for f in range(0, 50, 1):
for f in range(0, n_files, 1):
    if (f%10 == 0):
        print('  processing f %s of %s ' %(f, n_files)) 

    #lat_lon_temp = []
    lat_temp = []
    lon_temp = []
    ele_temp = []
    dt_temp  = []

    gpx_file_temp = ingest_file_list[f]
    # read GPX file
    with open(gpx_file_temp, 'r') as file:
        gpx = gpxpy.parse(file)
        for track in gpx.tracks:
            for segment in track.segments:
                for point in segment.points:
                    #lat_lon_temp.append([point.latitude, point.longitude])
                    lon_temp.append([point.longitude])
                    lat_temp.append([point.latitude])
                    ele_temp.append(point.elevation)
                    dt_temp.append(point.time) # convert time to timestamps (s)
          
    #lat_lon_temp = np.array(lat_lon_temp)  # [deg, deg]
    lon_temp = np.array(lon_temp) 
    lat_temp = np.array(lat_temp) 
    ele_temp = np.array(ele_temp) 
    dt_temp  = np.array( dt_temp) 

    n_points = len(lon_temp)
    #print('    read %s points ' %(n_points)) 
    n_points_old = n_points

    # use Ramer–Douglas–Peucker algorithm to reduce the number of trackpoints
    if (use_RDP):
        temp_array = np.hstack([lat_temp, lon_temp, np.arange(0, n_points, 1).reshape(-1, 1)])
        temp_array_new = RDP(temp_array, epsilon) # remove trackpoints less than epsilon meters away from the new track
        index = temp_array_new[:,2].astype(int) # hack
        ele_temp = np.squeeze(ele_temp[index])
        lon_temp = np.squeeze(lon_temp[index])
        lat_temp = np.squeeze(lat_temp[index])
        dt_temp  = np.squeeze( dt_temp[index])
        n_points = len(lon_temp)
        print('    reduced points from %s to %s ' %(n_points_old, n_points)) 
                
    # create GeoJSON feature collection
    features = []
    for i in np.arange(1, n_points):
        # note csmith - not sure which way this should go 
        line = geojson.LineString([(lon_temp[i-1], lat_temp[i-1]), (lon_temp[i], lat_temp[i])]) 
        #line = geojson.LineString([(lat_temp[i-1], lon_temp[i-1]), (lat_temp[i], lon_temp[i])]) 
        #line = geojson.LineString([(lat_lon_data[i-1, 1], lat_lon_data[i-1, 0]), (lat_lon_data[i, 1], lat_lon_data[i, 0])]) # (lon,lat) to (lon,lat) format
        #feature = geojson.Feature(geometry=line, properties={'elevation': float('%.1f'%elevation_data[i]), 'slope': float('%.1f'%slope_data[i]), 'speed': float('%.1f'%speed_data[i])})
        feature = geojson.Feature(geometry=line, properties={'date': ('%s'%dt_temp[i])})
        features.append(feature)
    feature_collection = geojson.FeatureCollection(features)

    file_name = os.path.basename(gpx_file_temp.strip('.gpx'))
    # write geojson file
    geojson_write_file = gpx_file_temp.replace(file_name,dt_temp[0].strftime('%Y-%m-%d_%H-%M')).replace(dir_gpx,dir_geojson).replace('.gpx','.geojson')        
    print('    geojson_write_file is %s ' %(geojson_write_file))
    with open(geojson_write_file, 'w') as file:
        geojson.dump(feature_collection, file)

    # rename and archive gpx file 
    gpx_file_name_archive = gpx_file_temp.replace(file_name,dt_temp[0].strftime('%Y-%m-%d_%H-%M'))
    print('    gpx_file_name_archive is %s ' %(gpx_file_name_archive))
    if (' ' in gpx_file_temp):
        temp_command = 'mv -f "'+gpx_file_temp+'" '+gpx_file_name_archive
    else:
        temp_command = 'mv -f '+gpx_file_temp+' '+gpx_file_name_archive
        
    if (f == 0):
        lat_all = lat_temp
        lon_all = lon_temp
        ele_all = ele_temp
        dt_all  = dt_temp
    else: 
        lat_all = np.hstack([lat_all, lat_temp])
        lon_all = np.hstack([lon_all, lon_temp])
        ele_all = np.hstack([ele_all, ele_temp])
        dt_all  = np.hstack([ dt_all,  dt_temp])
    del lon_temp, lat_temp, ele_temp, dt_temp 
    n_points_all = len(lat_all)
    #print('  %s total points ' %(n_points_all))    
    

  processing f 0 of 263 
    reduced points from 287 to 181 
    geojson_write_file is data/geojson/2020-02-06_01-14.geojson 
    gpx_file_name_archive is data/gpx/2020-02-06_01-14.gpx 
    reduced points from 756 to 495 
    geojson_write_file is data/geojson/2020-02-29_21-57.geojson 
    gpx_file_name_archive is data/gpx/2020-02-29_21-57.gpx 
    reduced points from 640 to 411 
    geojson_write_file is data/geojson/2020-02-15_20-50.geojson 
    gpx_file_name_archive is data/gpx/2020-02-15_20-50.gpx 
    reduced points from 101 to 66 
    geojson_write_file is data/geojson/2020-02-09_15-33.geojson 
    gpx_file_name_archive is data/gpx/2020-02-09_15-33.gpx 
    reduced points from 850 to 562 
    geojson_write_file is data/geojson/2020-03-07_17-53.geojson 
    gpx_file_name_archive is data/gpx/2020-03-07_17-53.gpx 
    reduced points from 879 to 584 
    geojson_write_file is data/geojson/2020-02-08_19-03.geojson 
    gpx_file_name_archive is data/gpx/2020-02-08_19-03.gpx 
    reduce

    reduced points from 357 to 258 
    geojson_write_file is data/geojson/2019-10-28_18-54.geojson 
    gpx_file_name_archive is data/gpx/2019-10-28_18-54.gpx 
    reduced points from 403 to 237 
    geojson_write_file is data/geojson/2019-11-06_19-20.geojson 
    gpx_file_name_archive is data/gpx/2019-11-06_19-20.gpx 
    reduced points from 642 to 377 
    geojson_write_file is data/geojson/2019-11-17_19-29.geojson 
    gpx_file_name_archive is data/gpx/2019-11-17_19-29.gpx 
    reduced points from 421 to 289 
    geojson_write_file is data/geojson/2019-11-27_19-48.geojson 
    gpx_file_name_archive is data/gpx/2019-11-27_19-48.gpx 
    reduced points from 378 to 233 
    geojson_write_file is data/geojson/2019-12-13_19-43.geojson 
    gpx_file_name_archive is data/gpx/2019-12-13_19-43.gpx 
    reduced points from 1814 to 428 
    geojson_write_file is data/geojson/2019-10-26_18-51.geojson 
    gpx_file_name_archive is data/gpx/2019-10-26_18-51.gpx 
    reduced points from 376 to 25

    reduced points from 331 to 201 
    geojson_write_file is data/geojson/2019-08-27_13-56.geojson 
    gpx_file_name_archive is data/gpx/2019-08-27_13-56.gpx 
    reduced points from 634 to 376 
    geojson_write_file is data/geojson/2019-09-01_14-50.geojson 
    gpx_file_name_archive is data/gpx/2019-09-01_14-50.gpx 
    reduced points from 456 to 315 
    geojson_write_file is data/geojson/2019-09-02_16-16.geojson 
    gpx_file_name_archive is data/gpx/2019-09-02_16-16.gpx 
    reduced points from 241 to 161 
    geojson_write_file is data/geojson/2019-09-04_16-55.geojson 
    gpx_file_name_archive is data/gpx/2019-09-04_16-55.gpx 
    reduced points from 354 to 243 
    geojson_write_file is data/geojson/2019-09-10_13-54.geojson 
    gpx_file_name_archive is data/gpx/2019-09-10_13-54.gpx 
    reduced points from 352 to 235 
    geojson_write_file is data/geojson/2019-09-13_16-09.geojson 
    gpx_file_name_archive is data/gpx/2019-09-13_16-09.gpx 
    reduced points from 542 to 386

    reduced points from 412 to 257 
    geojson_write_file is data/geojson/2019-01-20_18-50.geojson 
    gpx_file_name_archive is data/gpx/2019-01-20_18-50.gpx 
    reduced points from 347 to 236 
    geojson_write_file is data/geojson/2019-01-23_14-55.geojson 
    gpx_file_name_archive is data/gpx/2019-01-23_14-55.gpx 
    reduced points from 1086 to 801 
    geojson_write_file is data/geojson/2019-01-27_15-27.geojson 
    gpx_file_name_archive is data/gpx/2019-01-27_15-27.gpx 
    reduced points from 414 to 304 
    geojson_write_file is data/geojson/2019-02-23_00-04.geojson 
    gpx_file_name_archive is data/gpx/2019-02-23_00-04.gpx 
    reduced points from 680 to 449 
    geojson_write_file is data/geojson/2019-03-03_21-23.geojson 
    gpx_file_name_archive is data/gpx/2019-03-03_21-23.gpx 
    reduced points from 544 to 377 
    geojson_write_file is data/geojson/2019-02-24_15-53.geojson 
    gpx_file_name_archive is data/gpx/2019-02-24_15-53.gpx 
  processing f 160 of 263 
    re

    reduced points from 529 to 381 
    geojson_write_file is data/geojson/2018-12-09_16-33.geojson 
    gpx_file_name_archive is data/gpx/2018-12-09_16-33.gpx 
    reduced points from 637 to 441 
    geojson_write_file is data/geojson/2018-12-22_17-38.geojson 
    gpx_file_name_archive is data/gpx/2018-12-22_17-38.gpx 
    reduced points from 1345 to 975 
    geojson_write_file is data/geojson/2018-11-25_15-06.geojson 
    gpx_file_name_archive is data/gpx/2018-11-25_15-06.gpx 
    reduced points from 345 to 202 
    geojson_write_file is data/geojson/2018-12-27_16-48.geojson 
    gpx_file_name_archive is data/gpx/2018-12-27_16-48.gpx 
    reduced points from 351 to 237 
    geojson_write_file is data/geojson/2018-12-26_16-36.geojson 
    gpx_file_name_archive is data/gpx/2018-12-26_16-36.gpx 
    reduced points from 324 to 239 
    geojson_write_file is data/geojson/2018-12-28_16-02.geojson 
    gpx_file_name_archive is data/gpx/2018-12-28_16-02.gpx 
  processing f 210 of 263 
    re

    reduced points from 459 to 351 
    geojson_write_file is data/geojson/2018-07-26_00-28.geojson 
    gpx_file_name_archive is data/gpx/2018-07-26_00-28.gpx 
    reduced points from 351 to 237 
    geojson_write_file is data/geojson/2018-08-11_15-02.geojson 
    gpx_file_name_archive is data/gpx/2018-08-11_15-02.gpx 
    reduced points from 561 to 389 
    geojson_write_file is data/geojson/2018-09-19_00-44.geojson 
    gpx_file_name_archive is data/gpx/2018-09-19_00-44.gpx 
    reduced points from 462 to 320 
    geojson_write_file is data/geojson/2018-09-08_15-09.geojson 
    gpx_file_name_archive is data/gpx/2018-09-08_15-09.gpx 
    reduced points from 450 to 312 
    geojson_write_file is data/geojson/2018-09-29_22-53p.geojson 
    gpx_file_name_archive is data/gpx/2018-09-29_22-53p.gpx 
  processing f 260 of 263 
    reduced points from 418 to 282 
    geojson_write_file is data/geojson/2018-10-02_23-54.geojson 
    gpx_file_name_archive is data/gpx/2018-10-02_23-54.gpx 
    r

In [6]:
print('%s total points ' %(n_points_all))    
n_time_visited = np.full([n_points_all], 1, dtype=int)



75752 total points 


In [7]:
# count and reduce nearby points
n = 10000
for n in range(0, n_points_all, 1):
    if (n%1000 == 0):
        print('  processing n %s of %s ' %(n, n_points_all)) 
    lat_temp = lat_all[n]
    lon_temp = lon_all[n]
    ele_temp = ele_all[n]
    #print(lat_temp, lon_temp, ele_temp)
    #dist_temp = (lat_all-lat_temp)**2.0 + (lon_all-lon_temp)**2
    #print(np.shape(dist_temp))
    #print(np.shape(dist_temp))
    #print(np.min(dist_temp))
    #print(np.max(dist_temp))
    #dist_temp

    if not (np.isnan(lon_temp)):
        dist_temp = calc_dist_between_one_point_to_all_points(lon_temp, lat_temp, lon_all, lat_all)
        index_close = np.argwhere(dist_temp < dist_min_aggregate_points)
        n_nearby_points = len(index_close)
        #print('    found %s nearby points ' %(n_nearby_points)) 
        if (n_nearby_points > 1):
            lon_avg = np.mean(lon_all[index_close])
            lat_avg = np.mean(lat_all[index_close])
            ele_avg = np.mean(ele_all[index_close])
            #print(lat_temp, lon_temp, ele_temp)
            #print(lat_avg, lon_avg, ele_avg)
            #print(index_close)
            #print(lon_all[index_close])
            #print(lat_all[index_close])
            #print(ele_all[index_close])
            lon_all[index_close] = np.nan
            lat_all[index_close] = np.nan
            ele_all[index_close] = np.nan
            lon_all[n] = lon_avg
            lat_all[n] = lat_avg
            ele_all[n] = ele_avg
            n_time_visited[n] = n_nearby_points
            del lon_avg, lat_avg, ele_avg
        del dist_temp, index_close, n_nearby_points
        

  processing n 0 of 75752 


/home/craigmatthewsmith/anaconda3/envs/env_gis/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in less


  processing n 1000 of 75752 
  processing n 2000 of 75752 
  processing n 3000 of 75752 
  processing n 4000 of 75752 
  processing n 5000 of 75752 
  processing n 6000 of 75752 
  processing n 7000 of 75752 
  processing n 8000 of 75752 
  processing n 9000 of 75752 
  processing n 10000 of 75752 
  processing n 11000 of 75752 
  processing n 12000 of 75752 
  processing n 13000 of 75752 
  processing n 14000 of 75752 
  processing n 15000 of 75752 
  processing n 16000 of 75752 
  processing n 17000 of 75752 
  processing n 18000 of 75752 
  processing n 19000 of 75752 
  processing n 20000 of 75752 
  processing n 21000 of 75752 
  processing n 22000 of 75752 
  processing n 23000 of 75752 
  processing n 24000 of 75752 
  processing n 25000 of 75752 
  processing n 26000 of 75752 
  processing n 27000 of 75752 
  processing n 28000 of 75752 
  processing n 29000 of 75752 
  processing n 30000 of 75752 
  processing n 31000 of 75752 
  processing n 32000 of 75752 
  processing n 33

In [8]:

print(np.nanmin(n_time_visited))
print(np.nanmax(n_time_visited))


1
68


In [9]:
n_points_all_old = n_points_all
#print(np.shape(lat_all))
mask = ~np.isnan(lat_all)
lon_all = lon_all[mask] 
lat_all = lat_all[mask] 
ele_all = ele_all[mask] 
n_time_visited = n_time_visited[mask] 
n_points_all = len(lat_all)
print('reduced points from %s to %s ' %(n_points_all_old, n_points_all))    


reduced points from 75752 to 44553 


In [10]:
#type(n_all)
#print(np.shape(n_all))
#n_all = np.array(n_all).astype(int)
#print(np.shape(n_all))


In [11]:
print(np.nanmin(n_time_visited))
print(np.nanmax(n_time_visited))
print(np.shape(lon_all))
print(np.shape(n_time_visited))


1
68
(44553,)
(44553,)


In [12]:
# only connect adjacent points, dont connect points from separate tracks 
features_thin = []

n = 1000
for n in range(1, n_points_all, 1):
    if (n%1000 == 0):
        print('  processing n %s of %s ' %(n, n_points_all)) 
    dist_temp = calc_dist_between_two_coords(lon_all[n], lat_all[n], lon_all[n-1], lat_all[n-1])
    n_temp = max(1, n_time_visited[n])
    if (dist_temp < dist_max_between_points_to_make_line):
        line = geojson.LineString([(lon_all[n], lat_all[n]), (lon_all[n-1], lat_all[n-1])])     
        feature = geojson.Feature(geometry=line, properties={'n_times': int('%.0f'%n_temp)})
        #feature = geojson.Feature(geometry=line, properties={'n_times': int('%.0f'%n_times)})
        features_thin.append(feature)
        del n_temp, line, feature
    
    
    



  processing n 1000 of 44553 
  processing n 2000 of 44553 
  processing n 3000 of 44553 
  processing n 4000 of 44553 
  processing n 5000 of 44553 
  processing n 6000 of 44553 
  processing n 7000 of 44553 
  processing n 8000 of 44553 
  processing n 9000 of 44553 
  processing n 10000 of 44553 
  processing n 11000 of 44553 
  processing n 12000 of 44553 
  processing n 13000 of 44553 
  processing n 14000 of 44553 
  processing n 15000 of 44553 
  processing n 16000 of 44553 
  processing n 17000 of 44553 
  processing n 18000 of 44553 
  processing n 19000 of 44553 
  processing n 20000 of 44553 
  processing n 21000 of 44553 
  processing n 22000 of 44553 
  processing n 23000 of 44553 
  processing n 24000 of 44553 
  processing n 25000 of 44553 
  processing n 26000 of 44553 
  processing n 27000 of 44553 
  processing n 28000 of 44553 
  processing n 29000 of 44553 
  processing n 30000 of 44553 
  processing n 31000 of 44553 
  processing n 32000 of 44553 
  processing n 33

In [13]:
feature_collection_thin = geojson.FeatureCollection(features_thin)
#file_name = 'master_thin.geojson'
file_name = 'master_thin_min_'+str(int(dist_min_aggregate_points))+'_max_'+str(int(dist_max_between_points_to_make_line))+'.geojson'
geojson_write_file = os.path.join(dir_work, file_name)
print('  geojson_write_file is %s ' %(geojson_write_file))
if os.path.isfile(geojson_write_file):
    temp_command = 'rm '+geojson_write_file
    os.system(temp_command)
with open(geojson_write_file, 'w') as file:
    geojson.dump(feature_collection_thin, file)



  geojson_write_file is /home/craigmatthewsmith/gps_tracks/master_thin_min_3_max_100.geojson 


In [ ]:
# dont use this 
# create lines from adjacent points 
# wrong way is to connect only nearbly points

features_thin2 = []

n = 1000
for n in range(0, n_points_all, 1):
    if (n%1000 == 0):
        print('  processing n %s of %s ' %(n, n_points_all)) 
    lat_temp = lat_all[n]
    lon_temp = lon_all[n]
    ele_temp = ele_all[n]
    n_temp   =   n_all[n]

    dist_temp = calc_dist_between_one_point_to_all_points(lon_temp, lat_temp, lon_all, lat_all)
    # note csmith - here not sure sure what dist_min for lines should be 
    index_close = np.argwhere(dist_temp < dist_min_adjacent_points_to_line)
    n_nearby_points = len(index_close)
    #print('    found %s nearby points' %(n_nearby_points))
    for i in range(0, n_nearby_points, 1):
        i_temp = index_close[i][0]
        #print ('    n %s, i %s, i_temp %s ' %(n, i, i_temp))
        if not (i_temp == n): # skip self 
            n_times = max(n_temp, n_all[i_temp]) # must be at least 1
            if (n_times == 0):
                n_times = 1
            line = geojson.LineString([(lon_temp, lat_temp), (lon_all[i_temp], lat_all[i_temp])]) 
            feature = geojson.Feature(geometry=line, properties={'n_times': int('%.0f'%n_times)})
            #feature = geojson.Feature(geometry=line, properties={'n_times': int('%.0f'%n_times)})
            features_thin2.append(feature)
            del n_times, line, feature
    


use_RDP = True
# use Ramer–Douglas–Peucker algorithm to reduce the number of trackpoints
if (use_RDP):
    epsilon = 1 # [m]
    tmp = np.hstack((lat_lon_data, np.arange(0, n_points).reshape((-1, 1)))) # hack
    tmp_new = RDP(tmp, epsilon) # remove trackpoints less than epsilon meters away from the new track
    index = tmp_new[:, 2].astype(int) # hack
    lat_lon_data   = lat_lon_data  [index,:]
    elevation_data = elevation_data[index]
    timestamp_data = timestamp_data[index]
    distance_data  = distance_data [index]
    slope_data     = slope_data    [index]
    speed_data     = speed_data    [index]

n_points = len(slope_data)
print('  read %s points ' %(n_points))    
    
# convert units
if use_SI:
    speed_data = speed_data*3.6 # m/s to km/h
else:
    speed_data = speed_data*2.236936 # m/s to mph

slope_data = abs(slope_data*100) # decimal to %

# create GeoJSON feature collection
features = []
for i in np.arange(1, n_points):
    #print('    processing %s of %s points ' %(i, n_points))    
    #print('    %s, %s, %s, %s ' %(lat_lon_data[i-1, 1], lat_lon_data[i-1, 0], lat_lon_data[i, 1], lat_lon_data[i, 0]))    
    try:
        line = geojson.LineString([(lat_lon_data[i-1, 1], lat_lon_data[i-1, 0]), (lat_lon_data[i, 1], lat_lon_data[i, 0])]) # (lon,lat) to (lon,lat) format
        feature = geojson.Feature(geometry=line, properties = {'elevation': float('%.1f'%elevation_data[i]), 'slope': float('%.1f'%slope_data[i]), 'speed': float('%.1f'%speed_data[i])})
        features.append(feature)
    except:
        print('    ERROR %s of %s points ' %(i, n_points))    

feature_collection = geojson.FeatureCollection(features)
